In [ ]:
# All subsequent cells summarized into one

SEGMENTS = ['ZUNU_vida-vessels.hdr', 'ZUNU_vida-sublobes.hdr', 'ZUNU_vida-lung.hdr', 'ZUNU_vida-airtree.hdr']
VidaDirectory = \
    r"Y:\E-Kelly-VidaSegmentals&Sublobes\3- 20Spiromics-10ForKelly-ForERS23\IA210305_H-17901_20151207_090246_20230524_121048_export"


# Find H-Number
import re
pattern = r'H-\d{5}'  # Regular expression pattern
matches = re.findall(pattern, VidaDirectory)  # Find all matches
HNUM = matches[0]  # Print the matched patterns
print('Found:', HNUM)


# Create VDysDir Dir
import os
VDysDir = os.path.join(VidaDirectory, 'VDysDir')
if not os.path.exists(VDysDir):
        os.makedirs(VDysDir)
print("VDysDir created in the VIDA folder")


# Search ImagesDirectory for H-NUM and find corresponding Case-#
ImagesDirectory = r"Y:\D-Images\SPIROMICS-SubStudy\2-Results-CheckedDoneTemp\1-Done"
def find_folder_with_hnum(path, HNUM):
    for root, dirs, files in os.walk(path):
        if root[-12:] == '1-Duplicates': continue
        print(root)
        for file in files:
            if HNUM in file:
                while True:
                    if "Case-" in os.path.basename(root):
                        return os.path.basename(root)
                    root = os.path.dirname(root)
                    if root == path or root == os.path.dirname(root):
                        break
    return None
result = find_folder_with_hnum(ImagesDirectory, HNUM)
if result: print("Found folder:", result)
else: print("No matching folder found.")
pattern = r'Case-\d{2}'  # Regular expression pattern
matches = re.findall(pattern, result)  # Find all matches
CNUM = matches[0]  # Print the matched patterns
print(CNUM+'\n')


# find all .nii.gz files, then pick the first! [print warning]
import glob
# Use glob to find all files with the extension ".nii.gz"
CASEPATH = os.path.join(ImagesDirectory, result)
IMGPATH = glob.glob(CASEPATH + '/**/*.nii.gz', recursive=True)
# Print the list of files
for IMPTH in IMGPATH:
    print(os.path.basename(IMPTH))
print('\nPicked the first as the target image')


# Copy image to VDysDir
import shutil
source_file = IMGPATH[0]
destination_folder = VDysDir
# Copy the file to the destination folder
shutil.copy(source_file, destination_folder)
print('Copied that image to VDysDir')


# List all files .nrrd, then find ones with the string 'label' > If several print warn > Pick last
# Use glob to find all files with the extension ".nrrd"
file_list = glob.glob(CASEPATH + '/**/*.nrrd', recursive=True)
# Filter the file list based on file names containing "Final" and "label"
LABELS = [file for file in file_list if 'Final' in file and 'label' in file]
if LABELS == []:
    print('WARNING: No Final-label.nrrd. FIX THAT PLEASE! Picking a non-label one.\n')
LABELS = [file for file in file_list if 'Final' in file]
for LABEL in LABELS:
    print(os.path.basename(LABEL))
LABELPATH = LABELS[-2]
print('\nPicked the last but one as the target labelmap')


# Copy nrrd labelmap to VDysDir
source_file = LABELPATH
destination_folder = VDysDir
# Copy the file to the destination folder
shutil.copy(source_file, destination_folder)
print('Copied that labelmap to VDysDir\n')


# List of segments = [..], Recenter them and put them in VDysDir
# re-center vessels and lung masks:
import SimpleITK as sitk
import numpy as np
imfn = os.path.join(VDysDir, os.path.basename(IMGPATH[0]))
iim = sitk.ReadImage(imfn)
for SEG in SEGMENTS:
    segfn = os.path.join(VidaDirectory, SEG)
    segim = sitk.ReadImage(segfn)
    snda = sitk.GetArrayFromImage(segim)
    snds = np.flip(snda, axis=0)
    fseg = sitk.GetImageFromArray(snds)
    fseg.CopyInformation(iim)
    outfn = SEG[0:-4] + '-recentered.nii.gz'
    sitk.WriteImage(fseg, os.path.join(VDysDir, outfn)) # flipped
    print('saved:', outfn)
print('\nDone.')

In [ ]:
path = "Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-15-MESAL-7018347-E3/MESAL-7018347-E3_FRC_80kV_0.75mm_Qr40d.nii.gz"
import SimpleITK as sitk
iim = sitk.ReadImage(path)
print(iim)


In [7]:
#Get array of all directories with files to be copied
#Could add a file to flag directories that have already had their files copied over to allow this code to be repeatedly run
import pathlib
import os

#Function for creating array of subdirectory names that match a given substring
def subdirMatch(directory, substring):
    matching = []
    for entry in os.listdir(directory):
        entry_path = os.path.join(directory, entry)
        if os.path.isdir(entry_path) and substring.lower() in entry.lower():
            matching.append(entry)
    return matching

#Function for finding subdirectory name that matches with given substring using recursion, 

def findSubdirs(directory, substring, )



dirPath = "Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done"
substr = "Case-"

caseDirList = subdirMatch(dirPath, substr)

#dirPathUnix = os.path()
#dirNamesList = str(os.listdir(dirPath))


#Stores all subdirectory names immediately within 'dirPath' - no recursion
for (dirpath, dirnames, filenames) in os.walk(dirNamesList):
    
    for currentDir in dirnames:
        #Ensures that the subdirectory is a completed 'Case' file before appending it to the array of directory names
        print(currentDir[4])
        #if(str(currentDir[:4]) == 'Case-'):
        if(check_filenames()):
            
            print("currentDir: "+ currentDir)
            newName = currentDir.replace('\\','/')
            dirNamesList.append(str(newName))
            print("dirNamesList: "+ dirNamesList)
        else:
            continue

#print(dirNamesList)

#Generate array of all subdirectories in 'dirPath' using 'scandir' (highly efficient method)
#Stores all absolute paths to each subdirectory
#pathList = [a.path for a in os.scandir(dirPath) if a.is_dir()]

i=0

useList = []

for dirs in dirNamesList:
    #print(dirs)
    #Ensures that the subdirectory is a completed 'Case' file before appending it to the array of paths to each subdirectory
    if(dirs[:4] == 'Case-'):
        #Convert path to Unix
        univPath = Path(dirs)
        dirs = dirs.replace("\\","/")
        useList.append(pathList[i])
    else:
        continue
    i+=1

#print(useList)




In [1]:
import os
import glob
import SimpleITK as sitk

#Recursion through directories, searches for files based on file name and extension, saves path to each matching file to one of two lists
def find_files_by_extension_and_prefix(root_folder, extension, prefix):
    segmentation_files = []
    final_files = []

    for dirpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            
            #S-Segmentation and S-Final (without 'label' in file name) in one list, S-Final with 'label' in file name in other list
            if filename.endswith(extension) and os.path.basename(dirpath).startswith(prefix):
                #".replace(os.path.sep, '/')" is used to corrent the backslashes in Windows' file paths to the Unix forwardslash
                full_path = os.path.join(dirpath, filename).replace(os.path.sep, '/')
                if filename.startswith("S-Segmentation"):
                    segmentation_files.append(full_path)
                elif filename.startswith("S-Final") and "label" not in filename:
                    segmentation_files.append(full_path)
                elif filename.startswith("S-Final") and "label" in filename:
                    final_files.append(full_path)



    return segmentation_files, final_files

#Copy files in segmentation & label lists generated by the 'find_files_by_extension_and_prefix' function
def copy_and_rename_files(source_dir, destination_dir):
    for root, _, files in os.walk(source_dir):
        for filename in files:
            source_file_path = os.path.join(root, filename)
            parent_dir = os.path.basename(root)
            new_filename = filename + '_' + parent_dir  # Appending parent directory name to the filename
            destination_file_path = os.path.join(destination_dir, new_filename)
            shutil.copy2(source_file_path, destination_file_path)


#=================================================================

#CODE TO GENERATE LABELMAP FROM SEGMENTATION FILES

#WIP (early attempt):

def convert_segmentation_to_labelmap(segmentation_file_path, labelmap_file_path):
    # Load the segmentation file
    segmentation = sitk.ReadImage(segmentation_file_path)

    # Convert the segmentation to a binary labelmap
    labelmap = sitk.BinaryThreshold(segmentation, lowerThreshold=1, upperThreshold=1)

    # Write the labelmap to the specified file path
    sitk.WriteImage(labelmap, labelmap_file_path)


#=================================================================

#=================================================================

#CODE TO COPY AND RENAME FILES TO NEW DIRECTORIES FOR MONAI

#WIP (early attempt):

def copy_and_rename_files(source_dir, destination_dir):
    for root, _, files in os.walk(source_dir):
        for filename in files:
            source_file_path = os.path.join(root, filename)
            parent_dir = os.path.basename(root)
            nameMatch = re.search(r'Case-(\d+)', parent_dir)  # Use regex to find the "Case-" followed by a sequence of digits
            if nameMatch:
                case_number = re.search(r'\d+', nameMatch)  # Use regex to find a sequence of digits in the parent directory name
                #case_number = nameMatch.group(1)  # Extract the matched sequence of digits
                new_filename = f"{filename}_Case-{case_number}"  # Append the found numbers to the filename
                destination_file_path = os.path.join(destination_dir, new_filename)
                shutil.copy2(source_file_path, destination_file_path)


#=================================================================

root_folder = "Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done"
extension = ".nrrd"
#Specifies directory name prefix to look for - will add '99-' later 
prefix = "88-"

segmentation_files, final_files = find_files_by_extension_and_prefix(root_folder, extension, prefix)

print("Files starting with 'S-Segmentation':")
for file in segmentation_files:
    print(file)

print("\nFiles starting with 'S-Final':")
for file in final_files:
    print(file)

Files starting with 'S-Segmentation':
Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-1-MESAL-5010098-E3/Markups/88-Nate/S-Final-1.seg.nrrd
Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-1-MESAL-5010098-E3/Markups/88-Nate/S-Final-3.seg.nrrd
Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-1-MESAL-5010098-E3/Markups/88-Nate/S-Segmentation-1.seg.nrrd
Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-1-MESAL-5010098-E3/Markups/88-Nate/S-Segmentation-2.seg.nrrd
Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-1-MESAL-5010098-E3/Markups/88-Nate/S-Segmentation-3.seg.nrrd
Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-1-MESAL-5010098-E3/Markups/88-Nate/S-Segmentation-5.seg.nrrd
Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-1-MESAL-5010098-E3/Markups/88-Nate/S-Segmentation-6.seg.n

In [13]:
A = "abclabel"
'label' not in A

False

In [ ]:
import pathlib
import os

dirPath = "Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-1-MESAL-5010098-E3/"






In [ ]:
import datetime

def list_nrrd_files_with_modified_date(root_folder):
    nrrd_files = []
    for folderpath, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.lower().endswith(".nrrd"):
                # Change to only get file name
                file_path = os.path.join(folderpath, filename)
                modified_date = os.path.getmtime(file_path)
                nrrd_files.append((filename, modified_date))
    return nrrd_files



root_folder = "Y:/public_resources/RA4-Project/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-1-MESAL-5010098-E3/"  # Replace this with the path to the root folder you want to traverse
nrrd_files_list = list_nrrd_files_with_modified_date(root_folder)

for file_path, modified_date in nrrd_files_list:
    print(f"File: {file_path}")
    last_modified_date = datetime.datetime.fromtimestamp(modified_date)
    print(f"Last Modified Date 1: {last_modified_date}")
    #print(f"Last Modified Date 2: {modified_date}")






In [47]:
# S-Final-*.nrrd, S-Segmentation-*.nrrd -> file patterns to look for, only pick the last one using [-1]

ABC = ['sdadas', 'fasfsfas', 'assdasdas']
ABC[-1]
ABC[-1][-1]

's'